In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiFPN(nn.Module):
    def __init__(self, in_channels, out_channels, num_levels=5):
        super(BiFPN, self).__init__()
        self.num_levels = num_levels
        self.w_fuse = nn.Parameter(torch.ones(self.num_levels, 4, requires_grad=True))

    def forward(self, inputs):
        """
        BiFPN forward pass.

        Args:
            inputs: List of feature maps from different levels.

        Returns:
            List of fused feature maps.
        """
        levels = len(inputs)
        assert levels == self.num_levels

        # Weighted feature fusion
        weights = self.w_fuse / torch.sum(self.w_fuse, dim=1, keepdim=True)
        P = [None] * levels

        for i in range(levels):
            w_top = weights[i, 0]
            w_bot = weights[i, 1]
            w_left = weights[i, 2]
            w_right = weights[i, 3]

            if i == 0:
                top = inputs[i + 1]
                left = None
            elif i == levels - 1:
                top = None
                left = inputs[i - 1]
            else:
                top = inputs[i + 1]
                left = inputs[i - 1]

            if i == 0:
                bot = None
                right = inputs[i]
            elif i == levels - 1:
                bot = inputs[i]
                right = None
            else:
                bot = inputs[i]
                right = inputs[i]

            if top is not None:
                top = F.interpolate(top, size=inputs[i].shape[-2:], mode='nearest')
            if left is not None:
                left = F.interpolate(left, size=inputs[i].shape[-2:], mode='nearest')
            if bot is not None:
                bot = inputs[i]
            if right is not None:
                right = inputs[i]

            P[i] = w_top * top + w_bot * bot + w_left * left + w_right * right

        return P

class SEAM(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEAM, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class GCNet(nn.Module):
    def __init__(self, in_channels):
        super(GCNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 1, kernel_size=1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, h, w = x.size()
        y = self.conv1(x)
        y = self.sigmoid(y)
        return x * y.expand_as(x)

class YOLOv10(nn.Module):
    def __init__(self, num_classes, anchors):
        super(YOLOv10, self).__init__()
        # ... (Define backbone and neck architecture) ...
        self.bifpn = BiFPN(in_channels, out_channels)
        self.seam = SEAM(out_channels)
        self.gcnet = GCNet(out_channels)
        # ... (Define head architecture) ...

    def forward(self, x):
        # ... (Backbone and neck forward pass) ...
        features = self.bifpn(features)
        features = [self.seam(f) for f in features]
        features = [self.gcnet(f) for f in features]
        # ... (Head forward pass) ...
        return outputs

# Example usage
model = YOLOv10(num_classes=80, anchors=[...])
# ... (Load weights, define optimizer, etc.) ...

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (images, targets) in enumerate(train_loader):
       # The code to follow

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

# Define the CNN model for character recognition
class CharacterRecognitionCNN(nn.Module):
    def __init__(self, num_classes, input_size=(28, 28)):
        super(CharacterRecognitionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * (input_size[0] // 4) * (input_size[1] // 4), 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * (x.size(2) * x.size(3)))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the trained CNN model
model = CharacterRecognitionCNN(num_classes=36)  # 36 for characters (0-9, A-Z)
model.load_state_dict(torch.load('character_recognition_model.pth'))
model.eval()

# Preprocessing function for character images
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Function to recognize a single character
def recognize_character(character_image):
    character_image = transform(character_image)
    character_image = character_image.unsqueeze(0)  # Add batch dimension
    output = model(character_image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# Example usage (assuming you have a list of detected characters)
detected_characters = ['A', 'B', 'C', '1', '2', '3']  # Replace with actual detected characters

recognized_plate = ""
for character in detected_characters:
    # Load the character image (replace with your image loading logic)
    character_image = Image.open(f'character_{character}.jpg')
    recognized_char = recognize_character(character_image)
    recognized_plate += str(recognized_char)

print("Recognized Plate:", recognized_plate)